# TensorFlow入门教程
https://segmentfault.com/a/1190000007484465

## 简介

TensorFlow是目前最流行的深度学习框架。我们先引用一段官网对于TensorFlow的介绍，来看一下Google对于它这个产品的定位。

TensorFlow™ is an open source software library for numerical computation using data flow graphs. Nodes in the graph represent mathematical operations, while the graph edges represent the multidimensional data arrays (tensors) communicated between them. The flexible architecture allows you to deploy computation to one or more CPUs or GPUs in a desktop, server, or mobile device with a single API.

上文并没有提到大红大紫的Deep Learning，而是聚焦在一个更广泛的科学计算应用领域。引文的关键词有：

- Numerical Computation：应用领域是数值计算，所以TensorFlow不仅能支持Deep Learning，还支持其他机器学习算法，甚至包括更一般的数值计算任务（如求导、积分、变换等）。
- Data Flow Graph：用graph来描述一个计算任务。
- Node：代表一个数学运算（mathmatical operations，简称ops），这里面包括了深度学习模型经常需要使用的ops。
- Edge：指向node的edge代表这个node的输入，从node引出来的edge代表这个node的输出，输入和输出都是multidimensional data arrays，即多维数组，在数学上又称之为tensor。这也是TensorFlow名字的由来，表示多维数组在graph中流动。
- CPUs/GPUs：支持CPU和GPU两种设备，支持单机和分布式计算。

TensorFlow提供多种语言的支持，其中支持最完善的是Python语言，因此本文将聚焦于Python API。

## Hello World
下面这段代码来自于TensorFlow官网的Get Started，展示了TensorFlow训练线性回归模型的能力。

In [5]:
import numpy as np
import tensorflow as tf

# Create 100 phony x, y data points in NumPy, y = x * 0.1 + 0.3
x_data = np.random.rand(100).astype(np.float32)
y_data = x_data * 0.1 + 0.3

# Try to find values for W and b that compute y_data = W * x_data + b
# (We know that W should be 0.1 and b 0.3, but TensorFlow will
# figure that out for us.)
W = tf.Variable(tf.random_uniform([1], -1, 1))
b = tf.Variable(tf.zeros([1]))
y = W * x_data + b

# Minimize the mean squared errors.
loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

# Before starting, initialize the variables.  We will 'run' this first.
init = tf.global_variables_initializer()

# Launch the graph.
with tf.Session() as sess:
    sess.run(init)
    # Fit the line.
    for step in range(201):
        sess.run(train)
        if step % 20 == 0:
            print(step, sess.run(W), sess.run(b))

0 [ 0.77606851] [-0.137935]
20 [ 0.31614694] [ 0.17623582]
40 [ 0.16693491] [ 0.26167357]
60 [ 0.12072794] [ 0.28813136]
80 [ 0.10641889] [ 0.29632461]
100 [ 0.10198776] [ 0.29886183]
120 [ 0.10061557] [ 0.29964754]
140 [ 0.10019064] [ 0.29989088]
160 [ 0.10005904] [ 0.29996622]
180 [ 0.10001828] [ 0.29998955]
200 [ 0.10000568] [ 0.29999676]


下面我们来剖析一下关键代码。TensorFlow的代码往往由两个部分组成：

- A construction phase, that assembles a graph 
- An execution phase that uses a session to execute ops in the graph.

Session是一个类，作用是把graph ops部署到Devices（CPUs/GPUs），并提供具体执行这些op的方法。

为什么要这么设计呢？考虑到Python运行性能较低，我们在执行numerical computing的时候，都会尽量使用非python语言编写的代码，比如使用NumPy这种预编译好的C代码来做矩阵运算。

在Python内部计算环境和外部计算环境（如NumPy）切换需要花费的时间称为overhead cost。对于一个简单运算，比如矩阵运算，从Python环境切换到Numpy，Numpy运算得到结果，再从Numpy切回Python，这个成本，比纯粹在Python内部做同类运算的成本要低很多。但是，一个复杂数值运算由多个基本运算组合而成，如果每个基本运算来一次这种环境切换，overhead cost就不可忽视了。为了减少来回的环境切换，TensorFlow的做法是，先在Python内定义好整个Graph，然后在Python外运行整个完整的Graph。因此TensorFlow的代码结构也就对应为两个阶段了。

## Build Graph

In [6]:
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.zeros([1]))

tf.Variable是TensorFlow的一个类，是取值可变的Tensor，构造函数的第一个参数是初始值initial_value。

initial_value: A Tensor, or Python object convertible to a Tensor, which is the initial value for the Variable.
tf.zeros(shape, dtype=tf.float32, name=None)是一个op，用于生成取值全是0的Constant Value Tensor。

tf.random_uniform(shape, minval=0, maxval=None, dtype=tf.float32, seed=None, name=None)是一个op，用于生成服从uniform distribution的Random Tensor。